In [2]:
%run Ability.ipynb
%run Status.ipynb

import pandas as pd

In [3]:
pets_df = pd.read_excel('pets_df.xlsx', index_col = [0,1])

for col in ['Trigger', 'Trigger_bywhom']:
    pets_df[col] = pets_df[col].str.split(', ')

for col in ['Target', 'Effect', 'Effect_from']:
    pets_df[col] = pets_df[col].str.split(', ').apply(lambda lst: [eval(s) for s in lst])

for col in ['Trigger_cond', 'Target_pos', 'Effect_info']:
    pets_df[col] = pets_df[col].apply(lambda x: x if x != x else eval(x))

In [4]:
pets_df.head()

Attack  Health  Id  Tier  Rollable  \
Name   Level                                       
Ant    1           2       1   0     1      True   
       2           4       2   0     1      True   
       3           6       3   0     1      True   
Beaver 1           3       2   3     1      True   
       2           6       4   3     1      True   

                                       Ability  Trigger Trigger_bywhom  \
Name   Level                                                             
Ant    1           Give a random friend +2/+1.  [Faint]         [Self]   
       2           Give a random friend +4/+2.  [Faint]         [Self]   
       3           Give a random friend +6/+3.  [Faint]         [Self]   
Beaver 1      Give 2 random friends +1 Health.   [Sell]         [Self]   
       2      Give 2 random friends +2 Health.   [Sell]         [Self]   

             Trigger_cond                             Target  ...  Target_pos  \
Name   Level                                                  ...               
Ant    1               {}  [<class '__main__.RandomFriend'>]  ...          ()   
       2               {}  [<class '__main__.RandomFriend'>]  ...          ()   
       3               {}  [<class '__main__.RandomFriend'>]  ...          ()   
Beaver 1               {}  [<class '__main__.RandomFriend'>]  ...          ()   
       2               {}  [<class '__main__.RandomFriend'>]  ...          ()   

                                        Effect                    Effect_from  \
Name   Level                                                                    
Ant    1      [<class '__main__.ModifyStats'>]  [<class '__main__.NoTarget'>]   
       2      [<class '__main__.ModifyStats'>]  [<class '__main__.NoTarget'>]   
       3      [<class '__main__.ModifyStats'>]  [<class '__main__.NoTarget'>]   
Beaver 1      [<class '__main__.ModifyStats'>]  [<class '__main__.NoTarget'>]   
       2      [<class '__main__.ModifyStats'>]  [<class '__main__.NoTarget'>]   

                              Effect_info Effect_repeat  N_charges  Pack1  \
Name   Level                                                                
Ant    1      {'add_atk': 2, 'add_hp': 1}             1        999   True   
       2      {'add_atk': 4, 'add_hp': 2}             1        999   True   
       3      {'add_atk': 6, 'add_hp': 3}             1        999   True   
Beaver 1                    {'add_hp': 1}             1        999   True   
       2                    {'add_hp': 2}             1        999   True   

              Pack2  Pack3  Custom  
Name   Level                        
Ant    1       True  False   False  
       2       True  False   False  
       3       True  False   False  
Beaver 1       True  False   False  
       2       True  False   False  

[5 rows x 21 columns]

In [5]:
foods_df = pd.read_excel('foods_df.xlsx', index_col = [0])

for col in ['Target', 'Effect', 'Effect_info']:
    foods_df[col] = foods_df[col].apply(lambda x: eval(x))

foods_df.head()

,Ability,Id,Tier,Rollable,Target,N_targets,Effect,Effect_info,Pack1,Pack2,Pack3,Custom
Name,,,,,,,,,,,,
Apple,Give a pet +1/+1.,0,1,True,<class '__main__.SelectedPet'>,-1,<class '__main__.ModifyStats'>,"{'add_atk': 1, 'add_hp': 1}",True,True,False,False
Bacon,Give a pet +2 Attack.,2,1,True,<class '__main__.SelectedPet'>,-1,<class '__main__.ModifyStats'>,{'add_atk': 2},False,False,False,True
Best Milk,Give a pet +3/+6.,103,1,False,<class '__main__.SelectedPet'>,-1,<class '__main__.ModifyStats'>,"{'add_atk': 3, 'add_hp': 6}",False,False,False,False
Better Milk,Give a pet +2/+4.,102,1,False,<class '__main__.SelectedPet'>,-1,<class '__main__.ModifyStats'>,"{'add_atk': 2, 'add_hp': 4}",False,False,False,False
Cookie,Give a pet +2/+2 until end of battle.,27,1,True,<class '__main__.SelectedPet'>,-1,<class '__main__.ModifyStats'>,"{'add_atk': 2, 'add_hp': 2, 'temp': True}",False,False,False,True


In [6]:
def get_species_by_tier(tier):
    possible = pets_df[pets_df.Rollable & (pets_df.Tier == tier)].index.get_level_values(0).values[::3]
    return np.random.choice(possible)

### Warning:

If making a custom pack, make sure to include summonables in dictionary's entry 0 (e.g. a dictionary with 'Sheep' must include 'Ram' in dictionary[0]).

Likewise, if including Cow or Crow, the food dictinary must include Milks and Chocolates, respectively.

In [29]:
custom_animal_dict = \
    {tier: pets_df[pets_df.Rollable & (pets_df.Tier == tier) & (pets_df.Custom)].index.get_level_values(0).values[::3] 
     for tier in [1, 2, 3, 4, 5, 6]}
custom_animal_dict[0] = pets_df[~pets_df.Rollable & (pets_df.Custom)].index.get_level_values(0).values[::3]

custom_food_dict = {tier: foods_df[foods_df.Custom & (foods_df.Tier == tier)].index.values for tier in [1, 2, 3, 4, 5, 6]}

In [21]:
faint_species = pets_df[pets_df.apply(lambda x: 'Faint' in x.Trigger and 'Self' in x.Trigger_bywhom, axis=1)]\
                    .index.get_level_values(0).values[::3]

pack1_animal_dict = {}
for tier in [1, 2, 3, 4, 5, 6]:
    pack1_animal_dict[tier] = \
        pets_df[pets_df.Rollable & (pets_df.Tier == tier) & (pets_df.Pack1)].index.get_level_values(0).values[::3]
pack1_animal_dict[0] = pets_df[~pets_df.Rollable & (pets_df.Pack1)].index.get_level_values(0).values[::3]
    
pack2_animal_dict = {}
for tier in [1, 2, 3, 4, 5, 6]:
    pack2_animal_dict[tier] = \
        pets_df[pets_df.Rollable & (pets_df.Tier == tier) & (pets_df.Pack2)].index.get_level_values(0).values[::3]
pack2_animal_dict[0] = pets_df[~pets_df.Rollable & (pets_df.Pack2)].index.get_level_values(0).values[::3]
    
pack3_animal_dict = {}
for tier in [1, 2, 3, 4, 5, 6]:
    pack3_animal_dict[tier] = \
        pets_df[pets_df.Rollable & (pets_df.Tier == tier) & (pets_df.Pack3)].index.get_level_values(0).values[::3]
pack3_animal_dict[0] = pets_df[~pets_df.Rollable & (pets_df.Pack3)].index.get_level_values(0).values[::3]
    
allpacks_animal_dict = {}
for tier in [1, 2, 3, 4, 5, 6]:
    allpacks_animal_dict[tier] = \
        pets_df[pets_df.Rollable & (pets_df.Tier == tier)].index.get_level_values(0).values[::3]
allpacks_animal_dict[0] = pets_df[~pets_df.Rollable].index.get_level_values(0).values[::3]
    
pack1_food_dict = {tier: foods_df[foods_df.Pack1 & (foods_df.Tier == tier)].index.values for tier in [1, 2, 3, 4, 5, 6, 0]}
    
pack2_food_dict = {tier: foods_df[foods_df.Pack2 & (foods_df.Tier == tier)].index.values for tier in [1, 2, 3, 4, 5, 6, 0]}

pack3_food_dict = {tier: foods_df[foods_df.Pack3 & (foods_df.Tier == tier)].index.values for tier in [1, 2, 3, 4, 5, 6, 0]}

allpacks_food_dict = {tier: foods_df[foods_df.Tier == tier].index.values for tier in [1, 2, 3, 4, 5, 6, 0]}